In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error
import joblib

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [2]:
VERSION = 5

In [3]:
import gradio as gr
import pandas as pd
import joblib
import os

MODEL_PATH = f'/kaggle/input/ship-exhaust/scikitlearn/default/{VERSION}/ship_emission_lgbm_minimal.pkl'

def load_prediction_assets(path):
    if not os.path.exists(path):
        return None
    return joblib.load(path)

assets = load_prediction_assets(MODEL_PATH)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MultiOutputRegressor from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:


def run_inference(df_input):
    if assets is None:
        return "ERROR: couldn't find file model .pkl"
    
    model = assets['model']
    feature_cols = assets['feature_cols']
    target_cols = assets['target_cols']
    encoders = assets.get('encoders', {})
    
    df_proc = df_input.copy()
    
    df_proc['is_man'] = df_proc['is_man'].astype(str).apply(
        lambda x: 1 if x.upper() in ['TRUE', '1', 'YES'] else 0
    ).astype(int)

    for col, encoder in encoders.items():
        if col in df_proc.columns:
            df_proc[col] = encoder.transform(df_proc[col].astype(str))

    cat_features = ['type', 'valve_type', 'is_man']
    for col in cat_features:
        if col in df_proc.columns:
            df_proc[col] = df_proc[col].astype('category')

    X = df_proc[feature_cols]
    preds = model.predict(X)
    
    df_result = pd.DataFrame(preds, columns=target_cols)
    pollutants = ['PM10', 'PM2.5', 'NOx', 'SOx', 'CO', 'HC', 'CO2', 'N2O', 'CH4']
    
    for p in pollutants:
        cols_to_sum = [f'E{i}_{p}' for i in range(1, 6)]
        
        if all(col in df_result.columns for col in cols_to_sum):
            df_result[f'Total_{p}'] = df_result[cols_to_sum].sum(axis=1)
            
    return pd.concat([df_input.reset_index(drop=True), df_result], axis=1)

In [5]:
def predict_manual(month, ship_type, year_ship, buoy, t_anchor, v_trip, v_man, v_max, p_main, p_aux, is_man, valve, rpm):
    data = [[month, ship_type, year_ship, buoy, t_anchor, v_trip, v_man, v_max, p_main, p_aux, is_man, valve, rpm]]
    cols = ['month', 'type', 'year_ship', 'buoy', 'time_anchor', 'v_trip', 'v_maneuver', 'v_max', 'P_main', 'P_aux', 'is_man', 'valve_type', 'rpm']
    df = pd.DataFrame(data, columns=cols)
    
    result = run_inference(df)
    display_cols = [c for c in result.columns]
    return  result, result[display_cols].iloc[0].to_dict()

def predict_file(file):
    if file is None: return None, None
    try:
        df = pd.read_csv(file.name)
    except:
        df = pd.read_excel(file.name)
    result = run_inference(df)
    output_path = "predicted_emissions.xlsx"
    result.to_excel(output_path, index=False)
    return result.head(10), output_path

In [6]:
with gr.Blocks(title="Ship Emission Predictor") as demo:
    gr.Markdown("# 🚢 SYSTEM PREDICT EMISSIONS OF SHIP")
    
    with gr.Tab("Manual Predict"):
        with gr.Row():
            with gr.Column():
                month = gr.Slider(1, 12, label="Month", value=1, step=1)
                ship_type = gr.Dropdown(['container_ship'], label="Loại tàu", value='container_ship')
                year_ship = gr.Number(label="Year of ship", value=2010)
                buoy = gr.Number(label="Buoy", value=0)
                is_man = gr.Radio(["TRUE", "FALSE"], label="Động cơ MAN?", value="FALSE")
                valve = gr.Radio(["C3", "SV"], label="Loại Van", value="C3")
            with gr.Column():
                p_main = gr.Number(label="P_main (kW)", value=25000)
                p_aux = gr.Number(label="P_aux (kW)", value=5000)
                rpm = gr.Number(label="RPM", value=110)
                v_max = gr.Number(label="V_max", value=15)
                v_trip = gr.Number(label="V_trip", value=12)
                v_man = gr.Number(label="V_maneuver", value=5)
            with gr.Column():
                t_anchor = gr.Number(label="t_anchor", value=200)
                # t_anchor = gr.Number(label="Thời gian neo đậu", interactive=False)
                # t_trip = gr.Number(label="Thời gian hành trình", value=0.25)
                # t_man = gr.Number(label="Thời gian điều động", value=6)
        btn_single = gr.Button("🚀 Submit")
   
        out_json = gr.JSON(label="Results")
        out_df = gr.DataFrame(label="Results")
        btn_single.click(predict_manual, 
                         inputs=[month, ship_type, year_ship, buoy, t_anchor, v_trip, v_man, v_max, p_main, p_aux, is_man, valve, rpm], 
                         outputs=[out_df, out_json])

    with gr.Tab("File Predict"):
        gr.Markdown("### Upload your file")
        file_input = gr.File(label="Choose")
        btn_file = gr.Button("Submit")
        
        with gr.Row():
            out_df = gr.DataFrame(label="Showign 10 line of head")
        with gr.Row():
            out_file = gr.File(label="Save")
        
        btn_file.click(predict_file, inputs=file_input, outputs=[out_df, out_file])

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://b6d8c789863d9431fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
